In [ ]:
! pip install datasets sentencepiece transformers evaluate rouge_score bert_score huggingface_hub sentencepiece
! pip3 install - U scikit-learn scipy matplotlib


In [1]:
!nvidia-smi

Sat Apr  1 20:44:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:17:00.0 Off |                    0 |
| N/A   44C    P0    40W / 250W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  Off  | 00000000:CA:00.0 Off |                    0 |
| N/A   

In [2]:
import os
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from sacrebleu import corpus_bleu
import torch
import torch.nn as nn
from transformers import pipeline


2023-04-01 20:44:19.251217: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-01 20:44:19.311312: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-01 20:44:22.299991: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from datasets import load_dataset

data_files = {"train": "./train.pkl",
              "test": "./test.pkl",
              "val": "./val.pkl"}

raw_dataset = load_dataset("pandas", data_files=data_files, cache_dir=None)

train_data = raw_dataset['train']
test_data = raw_dataset['test']
val_data = raw_dataset['val']


Found cached dataset pandas (/user/skunusot/.cache/huggingface/datasets/pandas/default-821c83f9439ee750/0.0.0/3ac4ffc4563c796122ef66899b9485a3f1a977553e2d2a8a318c72b8cc6f2202)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer, GPT2Config

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    model_name, model_name, tie_encoder_decoder=False).to(device)

model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id

model.config.max_length = 64
model.config.early_stopping = True
model.config.num_beams = 1
model.config.vocab_size = model.config.encoder.vocab_size

encoder_max_length = 150
decoder_max_length = 150
batch_size = 32


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relatio

In [5]:
def tokenize_function(examples):
    inputs = [ex["input_text"] for ex in examples['conversation']]
    targets = [ex["label"] for ex in examples['conversation']]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=encoder_max_length, truncation=True, padding='max_length'
    )
    return model_inputs


In [6]:
tokenized_datasets = raw_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=raw_dataset["train"].column_names,
)


Loading cached processed dataset at /user/skunusot/.cache/huggingface/datasets/pandas/default-821c83f9439ee750/0.0.0/3ac4ffc4563c796122ef66899b9485a3f1a977553e2d2a8a318c72b8cc6f2202/cache-506ae7fe20f54f37.arrow


Map:   0%|          | 0/6435 [00:00<?, ? examples/s]

Loading cached processed dataset at /user/skunusot/.cache/huggingface/datasets/pandas/default-821c83f9439ee750/0.0.0/3ac4ffc4563c796122ef66899b9485a3f1a977553e2d2a8a318c72b8cc6f2202/cache-ec10705ede74da17.arrow


In [21]:
!pip3 install -U scikit-learn scipy matplotlib


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 9.2 MB/s eta 0:00:0000:0100:01m


In [7]:
import evaluate
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")
import sklearn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


# implement the compute matrix to be used in evaluation the model
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    rouge_score = rouge.compute(predictions=pred_str, references=label_str),

    return {"bleu": round(corpus_bleu(pred_str, [label_str]).score, 4),
            "rouge": rouge_score,
           }


In [11]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./model",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size//2,
    gradient_accumulation_steps=3,
    save_strategy="no",
    predict_with_generate=True,
    do_eval=True,
    evaluation_strategy="epoch",
    do_train=True,
    logging_steps=500,
    save_steps=32965 // (batch_size * 2),
    warmup_steps=100,
    eval_steps=10,
    num_train_epochs=1,
    overwrite_output_dir=True,
    save_total_limit=0,
)


In [12]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer
)


In [13]:
trainer.train()


/projects/academic/courses/cse546s23/skunusot/anaconda3/envs/cuda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/projects/academic/courses/cse546s23/skunusot/anaconda3/envs/cuda/lib/python3.10/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model tra

Epoch,Training Loss,Validation Loss,Bleu,Rouge
0,No log,0.648833,0.279500,"({'rouge1': 0.07856616716316728, 'rouge2': 0.010171653614621886, 'rougeL': 0.07099994315596983, 'rougeLsum': 0.07096936536278382},)"


/projects/academic/courses/cse546s23/skunusot/anaconda3/envs/cuda/lib/python3.10/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Trainer is attempting to log a value of "({'rouge1': 0.07856616716316728, 'rouge2': 0.010171653614621886, 'rougeL': 0.07099994315596983, 'rougeLsum': 0.07096936536278382},)" of type <class 'tuple'> for key "eval/rouge" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=269, training_loss=1.746240665478334, metrics={'train_runtime': 827.5193, 'train_samples_per_second': 62.466, 'train_steps_per_second': 0.325, 'total_flos': 9282327357888000.0, 'train_loss': 1.746240665478334, 'epoch': 1.0})

In [14]:
# save the model
trainer._save("./trained_model/model")
tokenizer.save_pretrained("./trained_model/tokenizer")


('./trained_model/tokenizer/tokenizer_config.json',
 './trained_model/tokenizer/special_tokens_map.json',
 './trained_model/tokenizer/vocab.txt',
 './trained_model/tokenizer/added_tokens.json',
 './trained_model/tokenizer/tokenizer.json')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
model.push_to_hub("CC-BERT2BERT")
tokenizer.push_to_hub("CC-BERT2BERT")


In [16]:
results = trainer.evaluate()


Trainer is attempting to log a value of "({'rouge1': 0.07856616716316728, 'rouge2': 0.010171653614621886, 'rougeL': 0.07099994315596983, 'rougeLsum': 0.07096936536278382},)" of type <class 'tuple'> for key "eval/rouge" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
